<a href="https://colab.research.google.com/github/Elshraby/GradProj2/blob/main/15_May_PIML_01Attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Replace embeddings with real airfoil parameters: lift slope, stall angle, max Cl, drag bucket, etc.

Or: encode the full C_L(α) and C_D(α) curves as tabulated inputs (e.g., mini-networks or interpolators).

In [ ]:
# --- Google Colab-ready Physics-Informed Surrogate Model for H-Darrieus VAWT ---

# 🟦 STEP 1: Setup
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Set random seed
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# 🟦 STEP 2: Load Dataset
csv_path = 'Data.csv'  # Replace with your CSV
raw_df = pd.read_csv(csv_path)
raw_df.dropna(inplace=True)

# 🟦 STEP 3: Custom Dataset
class VAWTDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

        self.num_feature_cols = ['Number of Blades', 'Cord Length', 'Height',
                                 'Diameter', 'Pitch Angle', 'Freestream Velocity', 'Tip Speed Ratio']
        self.airfoil_col = 'Airfoil'
        self.target_col = 'Power Coefficient'

        self.num_features = self.df[self.num_feature_cols].values.astype(np.float32)
        airfoil_cat = self.df[self.airfoil_col].astype('category')
        self.airfoil_codes = airfoil_cat.cat.codes.values.astype(np.int64)
        self.airfoil_mapping = dict(enumerate(airfoil_cat.cat.categories))
        self.targets = self.df[self.target_col].values.astype(np.float32).reshape(-1, 1)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return {
            'num_features': torch.tensor(self.num_features[idx]),
            'airfoil_type': torch.tensor(self.airfoil_codes[idx]),
            'target': torch.tensor(self.targets[idx])
        }

In [ ]:
# 🟦 STEP 4: Physics Equation Function

def bem_cp_batch(num_feat, airfoil_slope=2*np.pi):
    N, c, _, D, pitch, U_inf, TSR = [num_feat[:,i] for i in range(7)]
    R = D / 2.0
    λ = TSR
    B = N
    thetas = torch.linspace(0, 2*np.pi, steps=32, device=num_feat.device)
    dθ = thetas[1] - thetas[0]
    cp_vals = []
    for i in range(num_feat.size(0)):
        integrand = []
        for θ in thetas:
            W = U_inf[i] * torch.sqrt(1 + 2*λ[i]*torch.cos(θ) + λ[i]**2)
            φ = torch.atan2(U_inf[i]*torch.sin(θ), λ[i]*U_inf[i] + U_inf[i]*torch.cos(θ))
            α = φ - pitch[i]*np.pi/180
            C_L = airfoil_slope * α
            C_D = 0.01 + 0.02*(C_L**2)
            L = 0.5 * 1.225 * W**2 * c[i] * C_L
            D_ = 0.5 * 1.225 * W**2 * c[i] * C_D
            T = L*torch.cos(φ) - D_*torch.sin(φ)
            integrand.append(T * (λ[i]*U_inf[i]) * R[i])
        integral = torch.stack(integrand).sum() * dθ
        P = B[i] * integral / (2*np.pi)
        P_ref = 0.5 * 1.225 * (np.pi*R[i]**2) * U_inf[i]**3
        cp_vals.append(P / P_ref)
    return torch.stack(cp_vals).unsqueeze(1)

# 🟦 STEP 5: Neural Network Model with Airfoil Embedding

class PhysicsInformedVAWT(nn.Module):
    def __init__(self, in_size, airfoil_vocab, emb_size=4, hidden=64):
        super().__init__()
        self.emb = nn.Embedding(airfoil_vocab, emb_size)
        self.net = nn.Sequential(
            nn.Linear(in_size+emb_size, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden//2),
            nn.ReLU(),
            nn.Linear(hidden//2, 1)
        )

    def forward(self, x_num, x_air):
        emb = self.emb(x_air)
        return self.net(torch.cat([x_num, emb], dim=1))

In [ ]:
# 🟦 STEP 6: Train Function with Physics Loss

def train_phys_informed(model, loader, num_epochs=200, α=0.1, lr=1e-3):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    mse = nn.MSELoss()
    model.train()
    for ep in range(num_epochs):
        total_loss = 0
        for batch in loader:
            x_num = batch['num_features']
            x_air = batch['airfoil_type']
            y_true = batch['target']

            y_pred = model(x_num, x_air)
            loss_data = mse(y_pred, y_true)
            cp_phys = bem_cp_batch(x_num)
            loss_phys = mse(y_pred, cp_phys)
            loss = loss_data + α*loss_phys

            opt.zero_grad(); loss.backward(); opt.step()
            total_loss += loss.item()
        if (ep+1)%20==0:
            print(f"Epoch {ep+1}: Loss = {total_loss/len(loader):.5f}")

In [ ]:
# 🟦 STEP 7: Train Model

dataset = VAWTDataset(raw_df)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
input_dim = len(dataset.num_feature_cols)
airfoil_vocab_size = int(dataset.airfoil_codes.max()) + 1

model = PhysicsInformedVAWT(input_dim, airfoil_vocab_size)
train_phys_informed(model, train_loader, num_epochs=300, α=0.1, lr=1e-3)

In [ ]:
# 🟦 STEP 8: Save Model

torch.save(model.state_dict(), 'vawt_physics_model.pth')
with open('vawt_model_meta.json', 'w') as f:
    import json
    json.dump({"input_dim": input_dim, "airfoil_vocab_size": airfoil_vocab_size}, f)

print("✅ Model saved successfully.")

In [ ]:
# 🟦 STEP 9: Predict on New Sample

def predict_cp(model, input_features, airfoil_name, airfoil_mapping):
    rev_map = {v:k for k,v in airfoil_mapping.items()}
    airfoil_code = rev_map[airfoil_name]
    model.eval()
    with torch.no_grad():
        x = torch.tensor([input_features], dtype=torch.float32)
        a = torch.tensor([airfoil_code], dtype=torch.int64)
        y = model(x, a)
    return y.item()

In [ ]:
# 🔍 Example:
sample = [3, 0.2, 0.5, 0.6, 0.0, 7.0, 1.2]  # example design vector
pred = predict_cp(model, sample, 'NACA0015', dataset.airfoil_mapping)
print(f"Predicted Cp: {pred:.4f}")

Now, Will need to change the angle of attack for each blade at each position per revolution for each blade. or just will compute like and I'm assuming ~ at the optimum value.

What will happen the unkown airfoils. should be excluded for this PINN model?